## 引進函式庫
Numpy



> Python處理資料串的能力很弱-->硬寫還是可以但就很硬，所以引用專門處理List和Array資料的函示庫Numpy做為補強，詳細用法網路上隨便找都可以找得到



Pandas


>pd跟np用法非常相似，都是處理list的好用套件，主要利用這個函式庫是因為他可以在讀取csv可以將標題部分直接read進來(當然用np讀進來自行code上去也行)


matplotlib


> 視覺化好幫手 報告神器 真心好用 內建就很精美 網路上更多模板套件有空自行去搜尋


> %matplotlib inline 直接print在編譯器上



%import sys



> 看版本用的 如果要看每個套件版本的話可能要找各自函式庫 大部分都是 套件._version_這種形式


sklearn --> Scikit Learn函式庫



> 包好的套件 神器 新手必用 網路一堆自稱AI大神都在用這個當教程，阿回想起來就是別人套件包好的東西根本 那些教程都只是教你按按鈕而已 --> 反過來說如果只是要做應用的話，別人寫好開源的就是超方便啦，記得掛*開源 *應用 這種提示避免部必要麻煩




joblib

> 打包/讀取model，好用的小套件






In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image

In [3]:
import warnings
warnings.filterwarnings("ignore") # 把warning關掉

In [4]:
import sys
print (sys.version) 
print (sys.version_info)

3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
sys.version_info(major=3, minor=7, micro=11, releaselevel='final', serial=0)


In [5]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler 
from sklearn.datasets import load_breast_cancer
from sklearn.pipeline import make_pipeline,Pipeline

from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, train_test_split
from sklearn.model_selection import learning_curve, validation_curve, cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.svm import SVC

from sklearn.ensemble import VotingClassifier

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

import sklearn.externals
import joblib

# Read data  
使用Pd直接讀標題，非常便利

head看你前面幾個data 確認用，
也可以用tail或print語法。

In [ ]:
#讀取
data = pd.read_csv('0717MLdatacsv.csv')
data.head()

# Label
把前面第一行Label轉成數字，避免python無法識別文字問題

***記得training data要把label給drop掉，帶著答案做機器學習訓練根本作弊

In [ ]:
# 編碼
classnumlize = {'Source.Name':{'B': 0, 'R': 1, 'P': 2,}}
data.replace(classnumlize,inplace = True)
data['Source.Name'].value_counts()
data.head()

In [ ]:
# 把Label單一抓出來，並且將data列drop掉
Label = data['Source.Name']
data.drop( ['Source.Name'],axis = 1,inplace = True)
Label.head()

#Features
做資料標準化(非必要)-->做資料拆分   

標準化是為了 
1.壓縮特徵空間複雜度
2.讓資料快速收斂
3.避免太可怕的bias產生(如Afeature=1-10是正常數值，但Bfeature=1-1000000000000都是正常，那綁在一起訓練就會有很誇張的bias，Afeature就變得再也不重要)--->有很多改善方法啦，比如附加特徵權重之類的，但最簡單的還是做標準化。


In [ ]:
#檢查feature
feature = pd.get_dummies(data)
feature.head()

In [ ]:
#標準化 
std=StandardScaler()
std.fit(feature)
feature_std=std.transform(feature)
#拆分
feature_train,feature_test,Label_train,Label_test = train_test_split(feature_std,Label,test_size=0.3, 
                     stratify=Label,random_state = 0)

#看資料到底好不好步驟 <-- 我不確定有沒有正確名詞

主要就是利用資料variance ratio做特徵主成分的列表，這個之後要做特徵貢獻表很有用，可以砍特徵絳維阿或是做資料再擷取等(目前我沒用 怕範圍開太大收不了尾XD)，跟統計有關可以去找一些工程統計的解說，因為很難用文字解說，去看算式就真的很好懂。

後續隨便train一個簡單的model(LR-->邏輯迴歸簡稱)去做CV(cross validation簡稱)，
主要目的是看資料的方差跟偏差的tradeoff，如果variance很大但bias很小，那就代表資料有著一定的準確度但是資料非常不集中，會導致很多驗證set的資料會超出原本預設的分類超平面外；另外如果反過來variance很小但bias很大，那代表資料雖然集中但是跟設定的分類平面或有一段落差，主要體現就是整體分類準確度會上不來，俗稱outliners搞事情；最爛狀況就是方差、偏差兩個都很爛，那就不用多講爛資料絕對訓練不出好model。整套權衡會往準確度越好，但又要使他成績曲線盡量收斂情況，這種資料在季器學習模型訓練上最有利。

(上面一大段結論) 好資料才會有好模型



In [ ]:
cov_mat = np.cov(feature_train.T)
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)

print('\nEigenvalues \n%s' % eigen_vals)

tot = sum(eigen_vals)
var_exp = [(i / tot) for i in sorted(eigen_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)


plt.bar(range(1, 33), var_exp, alpha=0.5, align='center',
        label='Individual explained variance')
plt.step(range(1, 33), cum_var_exp, where='mid',
         label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
pipe_lr = make_pipeline(LogisticRegression(penalty='l2', random_state=1,solver='lbfgs', max_iter=10000))

train_sizes, train_scores, test_scores =\
                learning_curve(estimator=pipe_lr,
                               X=feature_train,
                               y=Label_train,
                               train_sizes=np.linspace(0.1, 1.0, 10),
                               cv=15,
                               n_jobs=1)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_mean,
         color='blue', marker='o',
         markersize=5, label='Training accuracy')

plt.fill_between(train_sizes,
                 train_mean + train_std,
                 train_mean - train_std,
                 alpha=0.15, color='blue')

test_mean2 = sorted(test_mean)
plt.plot(train_sizes, test_mean2,
         color='green', linestyle='--',
         marker='s', markersize=5,
         label='Validation accuracy')

plt.fill_between(train_sizes,
                 test_mean2 + 0.25*test_std,
                 test_mean2 - 0.25*test_std,
                 alpha=0.15, color='green')

plt.grid()
plt.xlabel('Number of training examples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

# 簡單驗證後-->正式進入模型訓練

先對每個選用的模型做k fold交叉驗證，為什麼要做? 泛化能力指標阿模型好壞能力指標阿網路上有一堆論文或教學看你要相信哪個，我個人認為是這個就是一個很莫名其妙的驗證法，到底為什麼可以更這麼多變數的情況去度量不同模型的能力? 
但資工就常常用這種很玄的驗證法，所以大家都在用-->我就用。



*這邊提醒10-fold會產生10個驗證模型，有些網路範例會把收斂成效最好的那個直接提出來當作訓練模型(競賽裡面很常發生，為了追求成績不管各種現象的模型，引用請慎選)，但這樣做法是在必須要擁有"我擁有超多data 落掉10%完全沒差的狀況"，然而這種狀況也會被人詬病說是資料集中成機壞的就挑掉太獨斷，所以還是老老實實做平均成績後把所有data整起來丟到training裡面。 


In [ ]:
 #換另外一種命名方式 把它全部變成Value，將data拆分訓練集和測試集
X = feature.values
y = Label.values
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, 
                     stratify=y,random_state = 0)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
# K-fold CV for K = 3 (LogisticRegression)
kfold = StratifiedKFold(n_splits=3).split(X_train, y_train)
pipe_lr = make_pipeline(StandardScaler(),
            LogisticRegression(penalty='l2', C=10000,random_state=500, solver='lbfgs'))
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

plt.plot(scores,
         color='blue', marker='o',
         markersize=5, label='Validation accuracy')
plt.grid()
plt.xlabel('Number of K')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# K-fold CV for K = 10 (LogisticRegression)
kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)
pipe_lr = make_pipeline(StandardScaler(),
            LogisticRegression(penalty='l2', C=10000,random_state=50, solver='lbfgs'))
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

plt.plot(scores,
         color='blue', marker='o',
         markersize=5, label='Validation accuracy')
plt.grid()
plt.xlabel('Number of K')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# K-fold CV for K = 3 (DecisionTree)
kfold = StratifiedKFold(n_splits=3).split(X_train, y_train)
pipe_lr = make_pipeline(DecisionTreeClassifier(max_depth=3, criterion='entropy',random_state=500))
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

plt.plot(scores,
         color='blue', marker='o',
         markersize=5, label='Validation accuracy')
plt.grid()
plt.xlabel('Number of K')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# K-fold CV for K = 10 (DecisionTree)
kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)
pipe_lr = make_pipeline(DecisionTreeClassifier(max_depth=3, criterion='entropy',random_state=1))
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

plt.plot(scores,
         color='blue', marker='o',
         markersize=5, label='Validation accuracy')
plt.grid()
plt.xlabel('Number of K')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# K-fold CV for K = 10 (SVM-rbf)
kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)
pipe_svc = make_pipeline(StandardScaler(),SVC(kernel='rbf',gamma=10, C=1,random_state=500))
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_svc.fit(X_train[train], y_train[train])
    score = pipe_svc.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

plt.plot(scores,
         color='blue', marker='o',
         markersize=5, label='Validation accuracy')
plt.grid()
plt.xlabel('Number of K')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# K-fold CV for K = 3 (SVM-rbf)
kfold = StratifiedKFold(n_splits=3).split(X_train, y_train)
pipe_svc = make_pipeline(StandardScaler(),SVC(kernel='rbf',gamma=10, C=1,random_state=500))
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_svc.fit(X_train[train], y_train[train])
    score = pipe_svc.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

plt.plot(scores,
         color='blue', marker='o',
         markersize=5, label='Validation accuracy')
plt.grid()
plt.xlabel('Number of K')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# K-fold CV for K = 10 (SVM-sigmoid)
kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)
pipe_svc = make_pipeline(StandardScaler(),SVC(kernel='sigmoid',gamma=0.001,C=1000,random_state=0))
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_svc.fit(X_train[train], y_train[train])
    score = pipe_svc.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

plt.plot(scores,
         color='blue', marker='o',
         markersize=5, label='Validation accuracy')
plt.grid()
plt.xlabel('Number of K')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# K-fold CV for K = 3 (SVM-sigmoid)
kfold = StratifiedKFold(n_splits=3).split(X_train, y_train)
pipe_svc = make_pipeline(StandardScaler(),SVC(kernel='sigmoid',gamma=0.001,C=1000,random_state=20))
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_svc.fit(X_train[train], y_train[train])
    score = pipe_svc.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

plt.plot(scores,
         color='blue', marker='o',
         markersize=5, label='Validation accuracy')
plt.grid()
plt.xlabel('Number of K')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# K-fold CV for K = 3 (SVM-Poly)
kfold = StratifiedKFold(n_splits=3).split(X_train, y_train)
pipe_svc = make_pipeline(StandardScaler(),SVC(kernel='poly',C=0.001,gamma=0.001,degree=2,random_state=0))
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_svc.fit(X_train[train], y_train[train])
    score = pipe_svc.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

plt.plot(scores,
         color='blue', marker='o',
         markersize=5, label='Validation accuracy')
plt.grid()
plt.xlabel('Number of K')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# K-fold CV for K = 10 (SVM-Poly)
kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)
pipe_svc = make_pipeline(StandardScaler(),SVC(kernel='poly',C=0.001,gamma=0.001,degree=2,random_state=0))
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_svc.fit(X_train[train], y_train[train])
    score = pipe_svc.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

plt.plot(scores,
         color='blue', marker='o',
         markersize=5, label='Validation accuracy')
plt.grid()
plt.xlabel('Number of K')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# K-fold CV for K = 10 (SVM-linear)
kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)
pipe_svc = make_pipeline(StandardScaler(),SVC(kernel='linear',C=10,random_state=0))
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_svc.fit(X_train[train], y_train[train])
    score = pipe_svc.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

plt.plot(scores,
         color='blue', marker='o',
         markersize=5, label='Validation accuracy')
plt.grid()
plt.xlabel('Number of K')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# K-fold CV for K = 3 (SVM-linear)
kfold = StratifiedKFold(n_splits=3).split(X_train, y_train)
pipe_svc = make_pipeline(StandardScaler(),SVC(kernel='linear',C=10,random_state=0))
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_svc.fit(X_train[train], y_train[train])
    score = pipe_svc.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

plt.plot(scores,
         color='blue', marker='o',
         markersize=5, label='Validation accuracy')
plt.grid()
plt.xlabel('Number of K')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

# 調參 / Fune turn / Hyperparameter optimize (各種講法看你喜歡講哪種)

主要就是兩種方法

1. CV調 

優點:可視化 

缺點: 超過兩個參數就變得極難識別

還是可以ova那種形式兩個兩個配對調，會變得很麻煩(不過真的有人這樣做，為了填補論文頁數-->我朋友作法，這樣data會超多，教授看了超滿意，認為學生在教授自己不懂的AI領域做了很多很多嘗試、實驗、跟數據分析，所以我朋友大大推薦這種方法，論文一整個章節放圖片跟分析文字就30頁了)


2. 網格調

優點:  設定範圍內 設定範圍內 設定範圍內 的爆破迭代 全部列給你看

缺點: 吃效能、設定範圍要選好就是一門大學問

我這邊設定參考網路做法看大家大概都怎麼調 然後只調那些主要影響該模型參數(但實際上還有更多)，這邊有偷懶用10倍10倍log的方式調(怕真的窮舉電腦會燒掉)，所以說真的是找到最好的也只是 在我設定範圍內 找到最好的超參數組合。

分隔線-------------------

*有沒有其他方法又快又好的調參 -->有 ，關鍵字:Transfer Learning 
用別人搞好的(這算是近期熱門方法網路很多文章跟實作可以去找找)

In [ ]:
# CV調節 LR   (penalty='l2',solver='lbfgs')

pipe_lr = make_pipeline(StandardScaler(),LogisticRegression(penalty='l2', random_state=1,solver='lbfgs', max_iter=1000))

param_range = [0.0001 ,0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]
train_scores, test_scores = validation_curve(estimator=pipe_lr, X=X_train, y=y_train, 
                param_name='logisticregression__C', param_range=param_range,cv=10)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(param_range, train_mean, 
         color='blue', marker='o', 
         markersize=5, label='Training accuracy')

plt.fill_between(param_range, train_mean + train_std,
                 train_mean - train_std, alpha=0.15,
                 color='blue')

plt.plot(param_range, test_mean, 
         color='green', linestyle='--', 
         marker='s', markersize=5, 
         label='Validation accuracy')

plt.fill_between(param_range, 
                 test_mean ,
                 test_mean , 
                 alpha=0.15, color='green')

plt.grid()
plt.xscale('log')
plt.legend(loc='lower right')
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# CV調節 LR   (penalty='l2',solver='newton-cg')

pipe_lr = make_pipeline(StandardScaler(),LogisticRegression(penalty='l2', random_state=1,solver='newton-cg', max_iter=1000))

param_range = [0.0001 ,0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]
train_scores, test_scores = validation_curve(estimator=pipe_lr, X=X_train, y=y_train, 
                param_name='logisticregression__C', param_range=param_range,cv=10)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(param_range, train_mean, color='blue', marker='o', markersize=5, label='Training accuracy')
plt.fill_between(param_range, train_mean + train_std,train_mean - train_std, alpha=0.15,color='blue')
plt.plot(param_range, test_mean,color='green', linestyle='--', marker='s', markersize=5,label='Validation accuracy')
plt.fill_between(param_range, test_mean ,test_mean , alpha=0.15, color='green')

plt.grid()
plt.xscale('log')
plt.legend(loc='lower right')
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# CV調節 LR   (penalty='l2',solver='sag')

pipe_lr = make_pipeline(StandardScaler(),LogisticRegression(penalty='l2', random_state=1,solver='sag', max_iter=1000))

param_range = [0.0001 ,0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]
train_scores, test_scores = validation_curve(estimator=pipe_lr, X=X_train, y=y_train, 
                param_name='logisticregression__C', param_range=param_range,cv=10)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(param_range, train_mean, color='blue', marker='o', markersize=5, label='Training accuracy')
plt.fill_between(param_range, train_mean + train_std,train_mean - train_std, alpha=0.15,color='blue')
plt.plot(param_range, test_mean,color='green', linestyle='--', marker='s', markersize=5,label='Validation accuracy')
plt.fill_between(param_range, test_mean ,test_mean , alpha=0.15, color='green')

plt.grid()
plt.xscale('log')
plt.legend(loc='lower right')
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.ylim([0.3, 1.03])
plt.tight_layout()
plt.show()

In [ ]:
# GS調節DT
regressor = DecisionTreeClassifier(random_state=0)
parameters = {'max_depth': range(1, 10)}
scoring_fnc = make_scorer(accuracy_score)
kfold = KFold(n_splits=10)

grid = GridSearchCV(regressor, parameters, scoring_fnc, cv=kfold)
grid = grid.fit(X_train, y_train)
reg = grid.best_estimator_

print('best score: %f'%grid.best_score_)
print('best parameters:')
for key in parameters.keys():
    print('%s: %d'%(key, reg.get_params()[key]))

print('test score: %f'%reg.score(X_test, y_test))


In [ ]:
# GS調節Poly
pipe_svc = make_pipeline(StandardScaler(),SVC(random_state=1))
scorer = make_scorer(score_func=precision_score, 
                         pos_label=1, 
                         greater_is_better=True, 
                         average='micro')

c_gamma_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0] 
degree_range = [2,3,4]
param_grid = [{'svc__C': c_gamma_range, 
               'svc__gamma': c_gamma_range,
               'svc__degree' : degree_range,
               'svc__kernel': ['poly']}]

gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=10,
                  n_jobs=-1)
gs = gs.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(gs.best_params_)
print()
print("Grid scores on development set:")
print()
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))
print()    
print('-----------------------------------------------')     
print("Classification report:")
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print("Best train accuracy from gridsearch:")
print(gs.best_score_)
print("Best parameters from gridsearch:")
print(gs.best_params_)
clf = gs.best_estimator_
print('Test accuracy: %.3f' % clf.score(X_test, y_test))
print('-----------------------------------------------') 

In [ ]:
# GS調節SVM_RBF
pipe_svc = make_pipeline(StandardScaler(),SVC(random_state=1))
scorer = make_scorer(score_func=precision_score, 
                         pos_label=1, 
                         greater_is_better=True, 
                         average='micro')

c_gamma_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0] 
param_grid = [{'svc__C': c_gamma_range,
               'svc__gamma': c_gamma_range,
               'svc__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=10,
                  n_jobs=-1)
gs = gs.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(gs.best_params_)
print()
print("Grid scores on development set:")
print()
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))
print()    
print('-----------------------------------------------')     
print("Classification report:")
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print("Best train accuracy from gridsearch:")
print(gs.best_score_)
print("Best parameters from gridsearch:")
print(gs.best_params_)
clf = gs.best_estimator_
print('Test accuracy: %.3f' % clf.score(X_test, y_test))
print('-----------------------------------------------') 

In [ ]:
# GS調節SVM-Linear
pipe_svc = make_pipeline(StandardScaler(),SVC(random_state=10000))
scorer = make_scorer(score_func=precision_score, 
                         pos_label=1, 
                         greater_is_better=True, 
                         average='micro')

c_gamma_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0] 
param_grid = [{'svc__C': c_gamma_range,
               'svc__kernel': ['linear']}]

gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=10,
                  n_jobs=-1)
gs = gs.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(gs.best_params_)
print()
print("Grid scores on development set:")
print()
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))
print()    
print('-----------------------------------------------')     
print("Classification report:")
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print("Best train accuracy from gridsearch:")
print(gs.best_score_)
print("Best parameters from gridsearch:")
print(gs.best_params_)
clf = gs.best_estimator_
print('Test accuracy: %.3f' % clf.score(X_test, y_test))
print('-----------------------------------------------') 

In [ ]:
# GS調節SVM_sigmoid
pipe_svc = make_pipeline(StandardScaler(),SVC(random_state=2))
scorer = make_scorer(score_func=precision_score, 
                         pos_label=1, 
                         greater_is_better=True, 
                         average='micro')

c_gamma_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0] 
param_grid = [{'svc__C': c_gamma_range,
               'svc__gamma': c_gamma_range,
               'svc__kernel': ['sigmoid']}]

gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=10,
                  n_jobs=-1)
gs = gs.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(gs.best_params_)
print()
print("Grid scores on development set:")
print()
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))
print()    
print('-----------------------------------------------')     
print("Classification report:")
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print("Best train accuracy from gridsearch:")
print(gs.best_score_)
print("Best parameters from gridsearch:")
print(gs.best_params_)
clf = gs.best_estimator_
print('Test accuracy: %.3f' % clf.score(X_test, y_test))
print('-----------------------------------------------') 

#包在一起訓練法 (當然也可以分開訓練在儲存)

目的: 整齊好看，僅此而已

訓練完後，簡單用最終測試print一下結果，然後用joblib儲存訓練好模型

In [ ]:
clf1 = LogisticRegression(penalty='l2', C=0.1,solver='lbfgs',random_state=500)

clf2 = DecisionTreeClassifier(max_depth=3, criterion='entropy',random_state=500)

clf3 = SVC(kernel='rbf',gamma=10, C=1,random_state=500)

clf4 = SVC(kernel='sigmoid',gamma=0.001,C=1000,random_state=500)

clf5 = SVC(kernel='poly',C=0.0001,gamma=0.0001,degree=2,random_state=500)

clf6 = SVC(kernel='linear',C=10,random_state=500)


pipe1 = Pipeline([['sc', StandardScaler()],
                  ['clf', clf1]])
pipe3 = Pipeline([['sc', StandardScaler()],
                  ['clf', clf3]])
pipe4 = Pipeline([['sc', StandardScaler()],
                  ['clf', clf4]])
pipe5 = Pipeline([['sc', StandardScaler()],
                  ['clf', clf5]])
pipe6 = Pipeline([['sc', StandardScaler()],
                  ['clf', clf6]])

eclf1 = VotingClassifier(estimators=[('lr', pipe1), ('dt', clf2), ('svm_rbf', pipe3), ('svm_sig', pipe4),
                                     ('svm_poly', pipe5), ('svm_linear', pipe6)], voting='hard')
eclf2 = VotingClassifier(estimators=[('lr', pipe1), ('dt', clf2), ('svm_rbf', pipe3), ('svm_sig', pipe4),
                                     ('svm_poly', pipe5), ('svm_linear', pipe6)], voting='hard', 
                         weights=[1,3,4,1,1,2], flatten_transform=True)

clf_labels = ['LogisticRegression model','DecisionTree model','svm_RBF model','svm_Sigmoid model','svm_Poly model','svm_linear model']


pipe1.fit(X_train, y_train)
print('---LogisticRegression model---')
print('Training accuracy:', pipe1.score(X_train, y_train))
print('Test accuracy:', pipe1.score(X_test, y_test))

clf2.fit(X_train, y_train)
print('---DecisionTree model---')
print('Training accuracy:', clf2.score(X_train, y_train))
print('Test accuracy:', clf2.score(X_test, y_test))


pipe3.fit(X_train, y_train)
print('---svm_RBF model---')
print('Training accuracy:', pipe3.score(X_train, y_train))
print('Test accuracy:', pipe3.score(X_test, y_test))

pipe4.fit(X_train, y_train)
print('---svm_Sigmoid model---')
print('Training accuracy:', pipe4.score(X_train, y_train))
print('Test accuracy:', pipe4.score(X_test, y_test))

pipe5.fit(X_train, y_train)
print('---svm_Poly model---')
print('Training accuracy:', pipe5.score(X_train, y_train))
print('Test accuracy:', pipe5.score(X_test, y_test))

pipe6.fit(X_train, y_train)
print('---svm_linear model---')
print('Training accuracy:', pipe6.score(X_train, y_train))
print('Test accuracy:', pipe6.score(X_test, y_test))


eclf1.fit(X, y)
print('---Hard Majority Vote---')
print('Training accuracy:', eclf1.score(X_train, y_train))
print('Test accuracy:', eclf1.score(X_test, y_test))

eclf2.fit(X, y)
print('---Soft Majority Vote---')
print('Training accuracy:', eclf2.score(X_train, y_train))
print('Test accuracy:', eclf2.score(X_test, y_test))


In [ ]:
kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)

clf_labels = ['1.Logistic regression', '2.Decision tree', '3.svm_RBF', '4.svm_Sigmoid', 
              '5.svm_Polynomial kernal', '6.svm_Linear','7.Bagging ', '8.Majority weighted Vote']
scores = []

print('K-fold cross validation:\n')
for clf, label in zip([pipe1, clf2, pipe3, pipe4, pipe5, pipe6, eclf1, eclf2], clf_labels):
    scores = cross_val_score(estimator=clf,X=X_train,y=y_train,cv=10)
    print("cross validation score: %0.2f (+/- %0.2f) [%s]"% (scores.mean(), scores.std(), label))

In [ ]:
#儲存
joblib.dump(clf2, "Tool_Life_Classification_Model/0716DT_model.dat")
joblib.dump(pipe1, "Tool_Life_Classification_Model/0716LRM_model.dat")
joblib.dump(pipe3, "Tool_Life_Classification_Model/0716RBF_model.dat")
joblib.dump(pipe4, "Tool_Life_Classification_Model/0716Sig_model.dat")
joblib.dump(pipe5, "Tool_Life_Classification_Model/0716Poly_model.dat")
joblib.dump(pipe6, "Tool_Life_Classification_Model/0716Line_model.dat")

In [ ]:
joblib.dump(eclf1, "Tool_Life_Classification_Model/0805Mjv_model.dat")
joblib.dump(eclf2, "Tool_Life_Classification_Model/0805rMjv_model.dat")

In [ ]:
#自寫bagging 無調(調不好) 最後還是決定用套件搞
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
import numpy as np
import operator


class MajorityVoteClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, classifiers, vote='classlabel', weights=None):

        self.classifiers = classifiers
        self.named_classifiers = {key: value for key, value in _name_estimators(classifiers)}
        self.vote = vote
        self.weights = weights

    def fit(self, X, y):

        if self.vote not in ('probability', 'classlabel'):
            raise ValueError("vote must be 'probability' or 'classlabel'" "; got (vote=%r)" % self.vote)

        if self.weights and len(self.weights) != len(self.classifiers):
            raise ValueError('Number of classifiers and weights must be equal' '; got %d weights, %d classifiers' % (len(self.weights), len(self.classifiers)))

        self.lablenc_ = LabelEncoder()
        self.lablenc_.fit(y)
        self.classes_ = self.lablenc_.classes_
        self.classifiers_ = []
        for clf in self.classifiers:
            fitted_clf = clone(clf).fit(X, self.lablenc_.transform(y))
            self.classifiers_.append(fitted_clf)
        return self

    def predict(self, X):
        if self.vote == 'probability':
            maj_vote = np.argmax(self.predict_proba(X), axis=1)
        else: 
            predictions = np.asarray([clf.predict(X)
             for clf in self.classifiers_]).T
                 maj_vote = np.apply_along_axis( lambda x: np.argmax(np.bincount(x, weights=self.weights)), axis=1, arr=predictions)
        maj_vote = self.lablenc_.inverse_transform(maj_vote)
        return maj_vote

    def predict_proba(self, X):
        probas = np.asarray([clf.predict_proba(X)
         for clf in self.classifiers_]) 
            avg_proba = np.average(probas, axis=0, weights=self.weights)
        return avg_proba

    def get_params(self, deep=True):
        if not deep:
            return super(MajorityVoteClassifier, self).get_params(deep=False)
        else:
            out = self.named_classifiers.copy()
            for name, step in self.named_classifiers.items():
                for key, value in step.get_params(deep=True).items():
                    out['%s__%s' % (name, key)] = value
            return out

In [ ]:
mv_clf = MajorityVoteClassifier(classifiers=[pipe1, clf2, pipe3, pipe4, pipe5, pipe6])
clf_labels += ['Majority voting']
all_clf = [pipe1, clf2, pipe3, pipe4, pipe5, pipe6, mv_clf]

for clf, label in zip(all_clf, clf_labels):
    scores = cross_val_score(estimator=clf, X=X_train,y=y_train, cv=10)
    print("accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
#看一下包起來裡面在幹嘛
mv_clf.get_params()

In [ ]:
# 自適應增強
from sklearn.ensemble import AdaBoostClassifier

tree = DecisionTreeClassifier(criterion='entropy', 
                              max_depth=1,
                              random_state=10)

ada = AdaBoostClassifier(base_estimator=tree,
                         n_estimators=1000, 
                         learning_rate=0.1,
                         random_state=10)

tree = tree.fit(X_train, y_train)
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)

tree_train = accuracy_score(y_train, y_train_pred)
tree_test = accuracy_score(y_test, y_test_pred)
print('Decision tree train/test accuracy %.14f/%.14f'
      % (tree_train, tree_test))

ada = ada.fit(X_train, y_train)
y_train_pred = ada.predict(X_train)
y_test_pred = ada.predict(X_test)

ada_train = accuracy_score(y_train, y_train_pred) 
ada_test = accuracy_score(y_test, y_test_pred) 
print('AdaBoost train/test accuracy %.14f/%.14f'
      % (ada_train, ada_test))

In [ ]:
joblib.dump(ada, "Tool_Life_Classification_Model/0807adaboost_model.dat")